In [1]:
import argilla as rg
import pandas as pd
from tqdm.auto import tqdm
from cleaner import Cleaner, Corretor

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rg.init(
        api_url='http://procyon.tce.pi.gov.br:6902',
        api_key='owner.apikey',
        workspace='victor_silva'
    )

/usr/local/lib/python3.8/dist-packages/argilla/client/client.py:170: UserWarning: You're connecting to Argilla Server 1.12.1 using a different client version (1.13.2).
This may lead to potential compatibility issues during your experience.
To ensure a seamless and optimized connection, we highly recommend aligning your client version with the server version.
  warnings.warn(


In [2]:
dados = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Objetos/objetosParaRotular.csv")
dados

,ID-LICITACAO,ID-ARQUIVO,objeto,TIPO_OBJETO
0,352975,338981,\n1.1. O objeto da presente licitação é a AQUI...,5.0
1,353289,340520,\n1.1 - Constitui o objeto da presente licitaç...,1.0
2,198167,143651,\n1.1 – Esta licitação tem por objeto a contra...,5.0
3,276775,283131,\n1.1. Constitui objeto do presente Edital a c...,1.0
4,463175,459049,\n1.1 Esta licitação tem por objeto a contrata...,3.0
...,...,...,...,...
6237,407651,418407,\n1.1 Constitui objeto da presente licitação a...,4.0
6238,506420,518417,\n1- A presente licitação tem por objeto a EVE...,1.0
6239,331016,310954,\n1.1. Antecedentes\n1.1.1. Constitui objeto d...,1.0
6240,232635,205330,"\n2.1.\nA presente licitação tem por objeto, R...",3.0


In [4]:
dados.rename(columns={"objeto": "text", "TIPO_OBJETO": "label"}, inplace=True)
dados

,ID-LICITACAO,ID-ARQUIVO,text,label
0,352975,338981,\n1.1. O objeto da presente licitação é a AQUI...,5.0
1,353289,340520,\n1.1 - Constitui o objeto da presente licitaç...,1.0
2,198167,143651,\n1.1 – Esta licitação tem por objeto a contra...,5.0
3,276775,283131,\n1.1. Constitui objeto do presente Edital a c...,1.0
4,463175,459049,\n1.1 Esta licitação tem por objeto a contrata...,3.0
...,...,...,...,...
6237,407651,418407,\n1.1 Constitui objeto da presente licitação a...,4.0
6238,506420,518417,\n1- A presente licitação tem por objeto a EVE...,1.0
6239,331016,310954,\n1.1. Antecedentes\n1.1.1. Constitui objeto d...,1.0
6240,232635,205330,"\n2.1.\nA presente licitação tem por objeto, R...",3.0


In [5]:
dados = dados[dados['label'] >= 1]
dados = dados[dados['label'] <= 4]
dados

,ID-LICITACAO,ID-ARQUIVO,text,label
1,353289,340520,\n1.1 - Constitui o objeto da presente licitaç...,1.0
3,276775,283131,\n1.1. Constitui objeto do presente Edital a c...,1.0
4,463175,459049,\n1.1 Esta licitação tem por objeto a contrata...,3.0
5,231901,203085,\n2.1 - A presente Licitação tem por objeto a ...,3.0
6,418522,430798,\nCONTRATAÇÃO DE PESSOA JURIDICA PARA O FORNEC...,4.0
...,...,...,...,...
6237,407651,418407,\n1.1 Constitui objeto da presente licitação a...,4.0
6238,506420,518417,\n1- A presente licitação tem por objeto a EVE...,1.0
6239,331016,310954,\n1.1. Antecedentes\n1.1.1. Constitui objeto d...,1.0
6240,232635,205330,"\n2.1.\nA presente licitação tem por objeto, R...",3.0


In [ ]:
def executaLimpeza(dataframe: pd.DataFrame, column: str, cased, accents) -> pd.DataFrame:
    """
    Executa a limpeza do DataFrame usando as classes Cleaner e Corretor.
    """
    sentencesDrop = []
    for indice in tqdm(dataframe.index, desc="Executando a Limpeza", colour='green'):
        texto = dataframe[column][indice]
        if isinstance(texto, str):
            texto = Cleaner().clear(texto)
            texto = Corretor(cased, accents).corrige_termos(texto)
            dataframe.at[indice, column] = texto
        else:
            sentencesDrop.append(indice)
    dataframe = dataframe.drop(sentencesDrop)
    return dataframe

In [ ]:
dados = executaLimpeza(dados, "text", cased=True, accents=True)

In [6]:
dados.dropna(subset=['text'], inplace=True)
dados.drop_duplicates(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

,ID-LICITACAO,ID-ARQUIVO,text,label
0,353289,340520,\n1.1 - Constitui o objeto da presente licitaç...,1.0
1,276775,283131,\n1.1. Constitui objeto do presente Edital a c...,1.0
2,463175,459049,\n1.1 Esta licitação tem por objeto a contrata...,3.0
3,231901,203085,\n2.1 - A presente Licitação tem por objeto a ...,3.0
4,418522,430798,\nCONTRATAÇÃO DE PESSOA JURIDICA PARA O FORNEC...,4.0
...,...,...,...,...
5731,407651,418407,\n1.1 Constitui objeto da presente licitação a...,4.0
5732,506420,518417,\n1- A presente licitação tem por objeto a EVE...,1.0
5733,331016,310954,\n1.1. Antecedentes\n1.1.1. Constitui objeto d...,1.0
5734,232635,205330,"\n2.1.\nA presente licitação tem por objeto, R...",3.0


In [7]:
dados['tamanho'] = dados['text'].apply(lambda x: len(x.split()))
dados['tamanho'].describe()

count    5736.000000
mean      190.575314
std       418.068716
min         1.000000
25%        58.000000
50%        98.000000
75%       244.250000
max      9627.000000
Name: tamanho, dtype: float64

In [8]:
dados = dados[dados['tamanho'] >= 10]
dados = dados[dados['tamanho'] <= 512]
dados

,ID-LICITACAO,ID-ARQUIVO,text,label,tamanho
0,353289,340520,\n1.1 - Constitui o objeto da presente licitaç...,1.0,192
1,276775,283131,\n1.1. Constitui objeto do presente Edital a c...,1.0,83
2,463175,459049,\n1.1 Esta licitação tem por objeto a contrata...,3.0,45
3,231901,203085,\n2.1 - A presente Licitação tem por objeto a ...,3.0,78
4,418522,430798,\nCONTRATAÇÃO DE PESSOA JURIDICA PARA O FORNEC...,4.0,21
...,...,...,...,...,...
5731,407651,418407,\n1.1 Constitui objeto da presente licitação a...,4.0,94
5732,506420,518417,\n1- A presente licitação tem por objeto a EVE...,1.0,88
5733,331016,310954,\n1.1. Antecedentes\n1.1.1. Constitui objeto d...,1.0,251
5734,232635,205330,"\n2.1.\nA presente licitação tem por objeto, R...",3.0,40


In [12]:
classes = ['Obras e serviços de engenharia', 'Material permanente', 'Material de consumo', 'Serviços']

In [10]:
mapping_labels = {1: classes[0], 2: classes[1], 3: classes[2], 4: classes[3]}
dados['label'] = dados['label'].map(mapping_labels)

In [14]:
records = []
for index in tqdm(dados.index, desc="Rotulando os dados", colour='yellow'):
    prediction = dados['label'][index]
    records.append(
        rg.TextClassificationRecord(
            id = index,
            text=dados['text'][index],
            status='Default',
            annotation = prediction,
            multi_label=True
        )
    )

Rotulando os dados:   0%|          | 0/5554 [00:00<?, ?it/s]/tmp/ipykernel_814529/4036634061.py:5: DeprecationWarning: Integer ids won't be supported in future versions. We recommend to start using strings instead. For datasets already containing integer values we recommend migrating them to avoid deprecation issues.See https://docs.argilla.io/en/latest/getting_started/installation/configurations/database_migrations.html#elasticsearch
  rg.TextClassificationRecord(
Rotulando os dados: 100%|██████████| 5554/5554 [00:00<00:00, 31613.80it/s]


In [15]:
settings = rg.TextClassificationSettings(classes)
rg.configure_dataset("objetostiposcased", settings, workspace="victor_silva")
rg.log(records, "objetostiposcased", workspace="victor_silva")

[06/07/24 11:37:55] WARNING  WARNING:argilla.datasets:This method is deprecated. Use                 ]8;id=676145;file:///usr/local/lib/python3.8/dist-packages/argilla/datasets/__init__.py\__init__.py]8;;\:]8;id=84237;file:///usr/local/lib/python3.8/dist-packages/argilla/datasets/__init__.py#80\80]8;;\
                             configure_dataset_settings instead.                                                   

/usr/local/lib/python3.8/dist-packages/rich/live.py:229: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

5554 records logged to ]8;id=370852;http://procyon.tce.pi.gov.br:6902/datasets/victor_silva/objetostiposcased\http://procyon.tce.pi.gov.br:6902/datasets/victor_silva/objetostiposcased]8;;\

BulkResponse(dataset='objetostiposcased', processed=5554, failed=0)